In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK


import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/ML_matrix_challenge/ML_matrix_challenge/matrix_two/ML_matrix_car'

/content/drive/My Drive/Colab Notebooks/ML_matrix_challenge/ML_matrix_challenge/matrix_two/ML_matrix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
#if isinstance(df[feat][0], list): continue - jezeli to jest lista to kontynuuj
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
#jezeli ten sufix jest feat to nadpiszmy to, co juz jest
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values
  #jezeli nie ma tworzymy je
  #to, co nam zwroci, chcemy przypisac do cechy

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) ) 
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')) ) 

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  #model = DecisionTreeRegressor(max_depth=5)
  #jesli ten zapis model = DecisionTreeRegressor(max_depth=5) zostawimy wewnatrz funkcji to uzna ten model za decydujacy, i nie przyjmie innych
#max_depth=5  im wieksza liczbe wpiszesz, tym lepszy wynik dostaniesz(teoretycznie),
#bo jak wrzucisz glebszy model drzewa decyzyjnego, to twoj model sie przyucza, a jak sie przyucza to czasem wychodza bzdury
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
#za pomoca tego wiersza powyzej odpalamy nasz model
#wpisujemy neg_mean_absolute_error zamiast mean_absolute_error, bo rezultat, ktory otrzymamy bedzie mial wartosc ujemna
  return np.mean(scores), np.std(scores)

In [10]:
feats =[ 'param_napęd_cat', 'param_rok-produkcji', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat', 'param_moc', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 'param_typ_cat', 'param_pojemność-skokowa', 
        'seller_name_cat', 'feature_wspomaganie-kierownicy_cat', 'param_model-pojazdu_cat', 'param_wersja_cat', 'param_kod-silnika_cat', 'feature_system-start-stop_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat', 'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat' ]

xgb_params = {
    'max_depth': 5,
#max_depth': 5 - mowimy przestrzen, ktorej poszukujemy (wykonujac krok nr1) znajduje sie gdzies miedzy 5, a 10
#te przestrzen przekazujemy do algorytmu, ktory bedzie to aktualizowac
#nastepnie algorytm robi kilka rzutow losowych, wyciaga wnioski
#i na podstawie tych wnioskow wykrywa w ktora strone najlepiej jest optymalizowac
#(w tym przypadku najlepiej = najmniejszy blad)
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model( xgb.XGBRegressor(**xgb_params), feats )

[22:53:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:53:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:53:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

#Hyperopt

In [22]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mae, score_std = run_model( xgb.XGBRegressor(**params), feats )
  # mean_mae, score_std...-tutaj trenujemy model uzywajac crosswalidacji

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}
  #calosc powyzej w tym wierszu to funkcja docelowa
  #I etap powyzej black box, ktory ma input i output
  #zwracamy slownik zlozony z II czesci
  #II-'status': STATUS_OK- potwierdza, ze udalo sie wyliczyc iteracje
  #I- 'loss': np.abs(mean_mae)- wartosc, na ktora oczekuje optymalizator
  #na inpucie jest black box, ma wplyw na to jakie parametry przekazuje, 
  #dalej nie wie, co my robimy z parametrami, oczekuje na wynik i po otrzymaniu stara sie go zmniejszyc
  #ale oczekuje od nas na odpowiedz jak dobrze nam poszlo z tymi parametrami

  #space
  #tutaj definiujemy dla naszego optymalizatora granice poszukiwania rozsadnych wartosci (sam tego nie wie, nie rozumie znaczenia np. max_depth)
xgb_reg_params = {
  'learning_rate':    hp.choice('learning_rate', np.arange(0.05,0.31, 0.05)),
  'max_depth':        hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
  'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
  'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
  'objective':        'reg:squarederror',
  'n_estimators':     100,
  'seed':             0
#pierwsze cztery parametry sa dynamiczne(zaczynajac od 'learning_rate') - podajemy przestrzen
#ostatnie dwa: n_estimators i seed sa statyczne
}

##run
#III etap: spinamy to, co bylo wczesniej
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=20)
best
#algo=tpe.suggest - algorytm optymalizacji
#max_evals=3 - liczba iteracji, jaka chcesz wykonac

Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.150000000000

{'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 6,
 'subsample': 0.65}